[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb)

# Semantic Search

In this walkthrough we will see how to use Pinecone for semantic search. To begin we must install the required prerequisite libraries:

In [21]:
!pip install -qU \
  pinecone-client==3.1.0 \
  pinecone-datasets==0.7.0 \
  sentence-transformers==2.2.2 \
  pinecone-notebooks==0.1.1

---

🚨 _Note: the above `pip install` is formatted for Jupyter notebooks. If running elsewhere you may need to drop the `!`._

---

## Data Download

In this notebook we will skip the data preparation steps as they can be very time consuming and jump straight into it with the prebuilt dataset from *Pinecone Datasets*. If you'd rather see how it's all done, please refer to [this notebook](https://github.com/pinecone-io/examples/blob/master/learn/search/semantic-search/semantic-search.ipynb).

Let's go ahead and download the dataset.

In [22]:
pip install --upgrade sentence-transformers

  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
Using cached sentence_transformers-3.3.1-py3-none-any.whl (268 kB)
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [1]:
pip install --upgrade huggingface_hub


In [7]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=14.0.0, but you have pyarrow 18.1.0 which is incompatible.
pinecone-datasets 0.7.0 requires pyarrow<12.0.0,>=11.0.0, but you have pyarrow 18.1.0 which is incompatible.
pylibcudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=1

In [4]:
from datasets import load_dataset

# Load a local JSON file
dataset = load_dataset('json', data_files='/content/generated_alumni_data.json', split='train')

# Convert the Dataset object to a pandas DataFrame for manipulation
df = dataset.to_pandas()

# Drop the 'metadata' column

# Rename the 'blob' column to 'metadata'
df.rename(columns={'blob': 'metadata'}, inplace=True)

# Select 80K rows from rows 240K to 320K
df = df.iloc[240_000:320_000]

# Display the first few rows
print(df.head())


Empty DataFrame
Columns: [id, name, email, phone, location, graduation_year, degree, stream, skills, current_employment, events_attended, tags]
Index: []


In [5]:
print(len(dataset))

100


## Creating an Index

Now the data is ready, we can set up our index to store it.

We begin by initializing our connection to Pinecone. To do this we need a [free API key](https://app.pinecone.io).

In [8]:
import os

# initialize connection to pinecone (orget API key at app.pinecone.io)
if not os.environ.get("PINECONE_API_KEY"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

In [6]:
from pinecone import Pinecone

api_key = "pcsk_HP4f4_SNXQ1GDvcbNH1tjnriAfGFWU2e9RY7KDxUdBc7fsPtr9UexezahTfyY2VK47k5F"

# configure client
pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [9]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index called `semantic-search-fast`. It's important that we align the index `dimension` and `metric` parameters with those required by the `MiniLM-L6` model.

In [10]:
index_name = 'semantic-search-fast2'

In [11]:
import time

existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,  # dimensionality of minilm
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Upsert the data:

In [26]:
from sentence_transformers import SentenceTransformer
import pinecone
from tqdm.auto import tqdm

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Assuming `dataset` is a list of 100 data entries similar to the one you provided
# Example dataset (replace this with your actual data)
dataset = [
    {
        "id": "1",
        "name": "David Lee",
        "email": "user1@example.com",
        "phone": "+42-918-3700",
        "location": {
            "city": "Tokyo",
            "state": "NY",
            "country": "Germany"
        },
        "graduation_year": 2001,
        "degree": "M.Sc",
        "stream": "Computer Science",
        "skills": [
            "Machine Learning",
            "Cloud Computing",
            "Python",
            "Finance Analysis"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "Alpha Solutions",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E685",
                "event_name": "Research Conference",
                "date": "2015-06-19",
                "location": "London, Japan"
            },
            {
                "event_id": "E302",
                "event_name": "Research Conference",
                "date": "2024-04-17",
                "location": "Toronto, Canada"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Mentor"
        ]
    },
    {
        "id": "2",
        "name": "Sophia Martinez",
        "email": "user2@example.com",
        "phone": "+59-666-4251",
        "location": {
            "city": "Berlin",
            "state": "MA",
            "country": "Australia"
        },
        "graduation_year": 2014,
        "degree": "PhD",
        "stream": "Computer Science",
        "skills": [
            "Data Analysis",
            "Finance Analysis",
            "Cloud Computing",
            "Web Development",
            "AI"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "FinanceCo",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E391",
                "event_name": "Finance Workshop",
                "date": "2024-06-01",
                "location": "London, Australia"
            },
            {
                "event_id": "E132",
                "event_name": "Finance Workshop",
                "date": "2018-12-24",
                "location": "Berlin, Germany"
            },
            {
                "event_id": "E734",
                "event_name": "Tech Symposium",
                "date": "2020-03-07",
                "location": "San Francisco, Australia"
            }
        ],
        "tags": [
            "AI Enthusiast"
        ]
    },
    {
        "id": "3",
        "name": "Emily Davis",
        "email": "user3@example.com",
        "phone": "+33-850-6089",
        "location": {
            "city": "Berlin",
            "state": "",
            "country": "Japan"
        },
        "graduation_year": 2005,
        "degree": "MBA",
        "stream": "Mechanical Engineering",
        "skills": [
            "Python",
            "AI",
            "Machine Learning",
            "Data Analysis",
            "Cloud Computing"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "Alpha Solutions",
            "location": "Berlin"
        },
        "events_attended": [
            {
                "event_id": "E233",
                "event_name": "Research Conference",
                "date": "2024-04-01",
                "location": "Sydney, USA"
            },
            {
                "event_id": "E170",
                "event_name": "Research Conference",
                "date": "2019-05-20",
                "location": "Toronto, Canada"
            },
            {
                "event_id": "E389",
                "event_name": "Annual Alumni Meet",
                "date": "2022-02-13",
                "location": "New York, Canada"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Developer"
        ]
    },
    {
        "id": "4",
        "name": "David Lee",
        "email": "user4@example.com",
        "phone": "+77-461-1761",
        "location": {
            "city": "Boston",
            "state": "",
            "country": "USA"
        },
        "graduation_year": 2000,
        "degree": "M.Sc",
        "stream": "Electronics",
        "skills": [
            "Risk Management",
            "Data Analysis",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Global Analytics",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E195",
                "event_name": "Annual Alumni Meet",
                "date": "2020-11-11",
                "location": "Sydney, Germany"
            },
            {
                "event_id": "E555",
                "event_name": "Annual Alumni Meet",
                "date": "2024-03-20",
                "location": "Berlin, Germany"
            },
            {
                "event_id": "E464",
                "event_name": "Research Conference",
                "date": "2017-10-09",
                "location": "Berlin, USA"
            }
        ],
        "tags": [
            "Mentor"
        ]
    },
    {
        "id": "5",
        "name": "Sarah Taylor",
        "email": "user5@example.com",
        "phone": "+89-941-1675",
        "location": {
            "city": "San Francisco",
            "state": "NY",
            "country": "Japan"
        },
        "graduation_year": 2011,
        "degree": "MBA",
        "stream": "Electronics",
        "skills": [
            "Machine Learning",
            "Cloud Computing",
            "Data Analysis",
            "Risk Management"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "FinanceCo",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E917",
                "event_name": "Research Conference",
                "date": "2021-09-20",
                "location": "New York, Canada"
            }
        ],
        "tags": [
            "Researcher"
        ]
    },
    {
        "id": "6",
        "name": "Michael Wilson",
        "email": "user6@example.com",
        "phone": "+14-986-7936",
        "location": {
            "city": "Tokyo",
            "state": "TX",
            "country": "UK"
        },
        "graduation_year": 2016,
        "degree": "PhD",
        "stream": "Physics",
        "skills": [
            "AI",
            "Web Development",
            "Machine Learning",
            "Risk Management",
            "Cloud Computing"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "Global Analytics",
            "location": "New York"
        },
        "events_attended": [
            {
                "event_id": "E718",
                "event_name": "Finance Workshop",
                "date": "2018-06-04",
                "location": "London, USA"
            },
            {
                "event_id": "E838",
                "event_name": "Research Conference",
                "date": "2016-09-07",
                "location": "London, USA"
            },
            {
                "event_id": "E779",
                "event_name": "Finance Workshop",
                "date": "2016-07-03",
                "location": "Boston, Canada"
            }
        ],
        "tags": [
            "Entrepreneur",
            "Researcher",
            "Developer"
        ]
    },
    {
        "id": "7",
        "name": "David Lee",
        "email": "user7@example.com",
        "phone": "+86-318-9118",
        "location": {
            "city": "New York",
            "state": "TX",
            "country": "Germany"
        },
        "graduation_year": 2002,
        "degree": "B.Tech",
        "stream": "Electronics",
        "skills": [
            "Finance Analysis",
            "Data Analysis",
            "Cloud Computing",
            "Python",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "Alpha Solutions",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E729",
                "event_name": "Annual Alumni Meet",
                "date": "2017-06-19",
                "location": "Sydney, Australia"
            },
            {
                "event_id": "E151",
                "event_name": "Finance Workshop",
                "date": "2018-02-21",
                "location": "Toronto, UK"
            },
            {
                "event_id": "E426",
                "event_name": "Research Conference",
                "date": "2019-03-20",
                "location": "San Francisco, UK"
            }
        ],
        "tags": [
            "Developer",
            "Researcher"
        ]
    },
    {
        "id": "8",
        "name": "Michael Wilson",
        "email": "user8@example.com",
        "phone": "+92-264-7871",
        "location": {
            "city": "Tokyo",
            "state": "MA",
            "country": "Japan"
        },
        "graduation_year": 2018,
        "degree": "MBA",
        "stream": "Finance",
        "skills": [
            "Web Development",
            "Cloud Computing",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "TechCorp",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E944",
                "event_name": "Finance Workshop",
                "date": "2015-07-01",
                "location": "San Francisco, USA"
            }
        ],
        "tags": [
            "Developer"
        ]
    },
    {
        "id": "9",
        "name": "Robert Brown",
        "email": "user9@example.com",
        "phone": "+58-977-7694",
        "location": {
            "city": "Sydney",
            "state": "TX",
            "country": "Canada"
        },
        "graduation_year": 2003,
        "degree": "MBA",
        "stream": "Biology",
        "skills": [
            "Data Analysis",
            "Web Development",
            "Finance Analysis",
            "Python",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "Global Analytics",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E303",
                "event_name": "Annual Alumni Meet",
                "date": "2022-02-04",
                "location": "London, Canada"
            },
            {
                "event_id": "E911",
                "event_name": "Annual Alumni Meet",
                "date": "2015-12-03",
                "location": "Toronto, Australia"
            }
        ],
        "tags": [
            "Researcher",
            "Developer"
        ]
    },
    {
        "id": "10",
        "name": "John Doe",
        "email": "user10@example.com",
        "phone": "+15-565-9071",
        "location": {
            "city": "New York",
            "state": "MA",
            "country": "Canada"
        },
        "graduation_year": 2017,
        "degree": "B.Tech",
        "stream": "Computer Science",
        "skills": [
            "Risk Management",
            "Python",
            "Web Development",
            "AI"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "TechCorp",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E430",
                "event_name": "Finance Workshop",
                "date": "2021-10-06",
                "location": "Tokyo, Germany"
            },
            {
                "event_id": "E873",
                "event_name": "Tech Symposium",
                "date": "2016-01-22",
                "location": "London, Australia"
            },
            {
                "event_id": "E762",
                "event_name": "Research Conference",
                "date": "2015-07-05",
                "location": "Boston, Australia"
            }
        ],
        "tags": [
            "Entrepreneur",
            "Finance Specialist"
        ]
    },
    {
        "id": "11",
        "name": "Alice Johnson",
        "email": "user11@example.com",
        "phone": "+16-844-8760",
        "location": {
            "city": "San Francisco",
            "state": "",
            "country": "Australia"
        },
        "graduation_year": 2004,
        "degree": "PhD",
        "stream": "Biology",
        "skills": [
            "Machine Learning",
            "Data Analysis",
            "Python",
            "Finance Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Innovatech",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E816",
                "event_name": "Research Conference",
                "date": "2021-05-17",
                "location": "Toronto, Canada"
            },
            {
                "event_id": "E469",
                "event_name": "Research Conference",
                "date": "2024-06-14",
                "location": "San Francisco, Australia"
            },
            {
                "event_id": "E392",
                "event_name": "Tech Symposium",
                "date": "2015-05-12",
                "location": "Sydney, Japan"
            }
        ],
        "tags": [
            "Mentor"
        ]
    },
    {
        "id": "12",
        "name": "Robert Brown",
        "email": "user12@example.com",
        "phone": "+90-914-8428",
        "location": {
            "city": "Sydney",
            "state": "CA",
            "country": "Canada"
        },
        "graduation_year": 2011,
        "degree": "MBA",
        "stream": "Electronics",
        "skills": [
            "Cloud Computing",
            "Python",
            "Machine Learning",
            "AI",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Global Analytics",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E305",
                "event_name": "Finance Workshop",
                "date": "2017-09-24",
                "location": "London, Canada"
            }
        ],
        "tags": [
            "Mentor",
            "AI Enthusiast",
            "Researcher"
        ]
    },
    {
        "id": "13",
        "name": "Emily Davis",
        "email": "user13@example.com",
        "phone": "+62-622-5011",
        "location": {
            "city": "San Francisco",
            "state": "CA",
            "country": "Canada"
        },
        "graduation_year": 2013,
        "degree": "B.Tech",
        "stream": "Computer Science",
        "skills": [
            "AI",
            "Machine Learning",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Global Analytics",
            "location": "Berlin"
        },
        "events_attended": [
            {
                "event_id": "E914",
                "event_name": "Annual Alumni Meet",
                "date": "2021-10-10",
                "location": "London, Australia"
            },
            {
                "event_id": "E193",
                "event_name": "Tech Symposium",
                "date": "2021-12-01",
                "location": "San Francisco, Canada"
            }
        ],
        "tags": [
            "AI Enthusiast",
            "Developer",
            "Researcher"
        ]
    },
    {
        "id": "14",
        "name": "James Anderson",
        "email": "user14@example.com",
        "phone": "+7-824-3696",
        "location": {
            "city": "Berlin",
            "state": "",
            "country": "Canada"
        },
        "graduation_year": 2002,
        "degree": "M.Sc",
        "stream": "Mechanical Engineering",
        "skills": [
            "Cloud Computing",
            "Python"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "Alpha Solutions",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E711",
                "event_name": "Research Conference",
                "date": "2015-10-03",
                "location": "Sydney, Australia"
            }
        ],
        "tags": [
            "Mentor",
            "Finance Specialist",
            "Entrepreneur"
        ]
    },
    {
        "id": "15",
        "name": "Jane Smith",
        "email": "user15@example.com",
        "phone": "+88-244-2751",
        "location": {
            "city": "San Francisco",
            "state": "NY",
            "country": "UK"
        },
        "graduation_year": 2006,
        "degree": "B.Tech",
        "stream": "Finance",
        "skills": [
            "Web Development",
            "Finance Analysis",
            "Risk Management",
            "Cloud Computing"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "TechCorp",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E940",
                "event_name": "Annual Alumni Meet",
                "date": "2016-06-25",
                "location": "Boston, Canada"
            },
            {
                "event_id": "E102",
                "event_name": "Research Conference",
                "date": "2016-02-07",
                "location": "London, Australia"
            },
            {
                "event_id": "E934",
                "event_name": "Annual Alumni Meet",
                "date": "2017-03-07",
                "location": "Boston, Canada"
            }
        ],
        "tags": [
            "Finance Specialist",
            "AI Enthusiast"
        ]
    },
    {
        "id": "16",
        "name": "Jane Smith",
        "email": "user16@example.com",
        "phone": "+74-707-6717",
        "location": {
            "city": "New York",
            "state": "TX",
            "country": "Australia"
        },
        "graduation_year": 2004,
        "degree": "M.Sc",
        "stream": "Computer Science",
        "skills": [
            "Finance Analysis",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "TechCorp",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E932",
                "event_name": "Finance Workshop",
                "date": "2023-08-08",
                "location": "Toronto, USA"
            },
            {
                "event_id": "E719",
                "event_name": "Finance Workshop",
                "date": "2017-03-03",
                "location": "Boston, USA"
            }
        ],
        "tags": [
            "Entrepreneur"
        ]
    },
    {
        "id": "17",
        "name": "Robert Brown",
        "email": "user17@example.com",
        "phone": "+11-727-6879",
        "location": {
            "city": "Berlin",
            "state": "TX",
            "country": "UK"
        },
        "graduation_year": 2016,
        "degree": "MBA",
        "stream": "Biology",
        "skills": [
            "AI",
            "Python",
            "Finance Analysis",
            "Risk Management",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Global Analytics",
            "location": "Tokyo"
        },
        "events_attended": [
            {
                "event_id": "E347",
                "event_name": "Research Conference",
                "date": "2019-05-07",
                "location": "London, Japan"
            },
            {
                "event_id": "E390",
                "event_name": "Tech Symposium",
                "date": "2019-07-06",
                "location": "Sydney, Japan"
            },
            {
                "event_id": "E344",
                "event_name": "Annual Alumni Meet",
                "date": "2021-04-12",
                "location": "New York, UK"
            }
        ],
        "tags": [
            "AI Enthusiast"
        ]
    },
    {
        "id": "18",
        "name": "Michael Wilson",
        "email": "user18@example.com",
        "phone": "+3-836-5680",
        "location": {
            "city": "New York",
            "state": "MA",
            "country": "UK"
        },
        "graduation_year": 2004,
        "degree": "B.Tech",
        "stream": "Electronics",
        "skills": [
            "Web Development",
            "Python",
            "AI"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "Alpha Solutions",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E715",
                "event_name": "Annual Alumni Meet",
                "date": "2024-11-24",
                "location": "Boston, USA"
            },
            {
                "event_id": "E970",
                "event_name": "Tech Symposium",
                "date": "2020-08-08",
                "location": "New York, Australia"
            },
            {
                "event_id": "E322",
                "event_name": "Tech Symposium",
                "date": "2015-10-14",
                "location": "San Francisco, Canada"
            }
        ],
        "tags": [
            "Developer",
            "Entrepreneur",
            "Mentor"
        ]
    },
    {
        "id": "19",
        "name": "Sophia Martinez",
        "email": "user19@example.com",
        "phone": "+13-455-8096",
        "location": {
            "city": "Sydney",
            "state": "NY",
            "country": "UK"
        },
        "graduation_year": 2018,
        "degree": "M.Sc",
        "stream": "Finance",
        "skills": [
            "Cloud Computing",
            "Machine Learning",
            "Web Development",
            "Data Analysis",
            "Risk Management"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "TechCorp",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E562",
                "event_name": "Tech Symposium",
                "date": "2018-11-19",
                "location": "Tokyo, Germany"
            },
            {
                "event_id": "E934",
                "event_name": "Annual Alumni Meet",
                "date": "2015-05-01",
                "location": "Tokyo, Australia"
            },
            {
                "event_id": "E163",
                "event_name": "Finance Workshop",
                "date": "2021-02-27",
                "location": "Toronto, UK"
            }
        ],
        "tags": [
            "AI Enthusiast"
        ]
    },
    {
        "id": "20",
        "name": "Robert Brown",
        "email": "user20@example.com",
        "phone": "+85-393-4345",
        "location": {
            "city": "Boston",
            "state": "TX",
            "country": "Canada"
        },
        "graduation_year": 2004,
        "degree": "M.Sc",
        "stream": "Finance",
        "skills": [
            "Cloud Computing",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Alpha Solutions",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E236",
                "event_name": "Annual Alumni Meet",
                "date": "2023-07-12",
                "location": "Sydney, USA"
            }
        ],
        "tags": [
            "Developer",
            "Finance Specialist"
        ]
    },
    {
        "id": "21",
        "name": "John Doe",
        "email": "user21@example.com",
        "phone": "+20-693-6321",
        "location": {
            "city": "New York",
            "state": "CA",
            "country": "USA"
        },
        "graduation_year": 2001,
        "degree": "MBA",
        "stream": "Mechanical Engineering",
        "skills": [
            "Python",
            "Data Analysis",
            "AI",
            "Machine Learning",
            "Risk Management"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "TechCorp",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E383",
                "event_name": "Annual Alumni Meet",
                "date": "2019-02-13",
                "location": "Berlin, Australia"
            },
            {
                "event_id": "E806",
                "event_name": "Research Conference",
                "date": "2023-05-28",
                "location": "Toronto, USA"
            }
        ],
        "tags": [
            "Researcher"
        ]
    },
    {
        "id": "22",
        "name": "Robert Brown",
        "email": "user22@example.com",
        "phone": "+44-879-4504",
        "location": {
            "city": "San Francisco",
            "state": "CA",
            "country": "Germany"
        },
        "graduation_year": 2003,
        "degree": "M.Sc",
        "stream": "Biology",
        "skills": [
            "Risk Management",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "FinanceCo",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E384",
                "event_name": "Tech Symposium",
                "date": "2015-03-09",
                "location": "Toronto, Canada"
            },
            {
                "event_id": "E411",
                "event_name": "Research Conference",
                "date": "2022-05-06",
                "location": "Berlin, USA"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Researcher",
            "Developer"
        ]
    },
    {
        "id": "23",
        "name": "Alice Johnson",
        "email": "user23@example.com",
        "phone": "+24-790-6275",
        "location": {
            "city": "Boston",
            "state": "NY",
            "country": "USA"
        },
        "graduation_year": 2007,
        "degree": "PhD",
        "stream": "Mechanical Engineering",
        "skills": [
            "Risk Management",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "TechCorp",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E493",
                "event_name": "Tech Symposium",
                "date": "2015-09-16",
                "location": "Sydney, UK"
            },
            {
                "event_id": "E138",
                "event_name": "Research Conference",
                "date": "2022-11-10",
                "location": "Berlin, Germany"
            },
            {
                "event_id": "E880",
                "event_name": "Tech Symposium",
                "date": "2019-12-21",
                "location": "Tokyo, Australia"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Mentor",
            "AI Enthusiast"
        ]
    },
    {
        "id": "24",
        "name": "John Doe",
        "email": "user24@example.com",
        "phone": "+11-249-4894",
        "location": {
            "city": "Berlin",
            "state": "NY",
            "country": "Canada"
        },
        "graduation_year": 2006,
        "degree": "MBA",
        "stream": "Finance",
        "skills": [
            "Python",
            "Cloud Computing",
            "Risk Management"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "TechCorp",
            "location": "Berlin"
        },
        "events_attended": [
            {
                "event_id": "E472",
                "event_name": "Annual Alumni Meet",
                "date": "2020-06-21",
                "location": "Boston, USA"
            },
            {
                "event_id": "E270",
                "event_name": "Tech Symposium",
                "date": "2019-09-22",
                "location": "Toronto, UK"
            }
        ],
        "tags": [
            "Developer",
            "Finance Specialist",
            "AI Enthusiast"
        ]
    },
    {
        "id": "25",
        "name": "Michael Wilson",
        "email": "user25@example.com",
        "phone": "+69-791-9173",
        "location": {
            "city": "New York",
            "state": "TX",
            "country": "USA"
        },
        "graduation_year": 2002,
        "degree": "MBA",
        "stream": "Finance",
        "skills": [
            "AI",
            "Data Analysis",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "FinanceCo",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E155",
                "event_name": "Tech Symposium",
                "date": "2018-02-22",
                "location": "Boston, USA"
            }
        ],
        "tags": [
            "Finance Specialist"
        ]
    },
    {
        "id": "26",
        "name": "Alice Johnson",
        "email": "user26@example.com",
        "phone": "+1-279-8980",
        "location": {
            "city": "Toronto",
            "state": "",
            "country": "Japan"
        },
        "graduation_year": 2001,
        "degree": "MBA",
        "stream": "Finance",
        "skills": [
            "Python",
            "Web Development",
            "AI"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Global Analytics",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E388",
                "event_name": "Finance Workshop",
                "date": "2023-10-21",
                "location": "Boston, Canada"
            },
            {
                "event_id": "E252",
                "event_name": "Annual Alumni Meet",
                "date": "2020-09-16",
                "location": "New York, UK"
            },
            {
                "event_id": "E605",
                "event_name": "Tech Symposium",
                "date": "2018-04-06",
                "location": "New York, UK"
            }
        ],
        "tags": [
            "Entrepreneur",
            "Researcher"
        ]
    },
    {
        "id": "27",
        "name": "Michael Wilson",
        "email": "user27@example.com",
        "phone": "+25-540-1539",
        "location": {
            "city": "San Francisco",
            "state": "CA",
            "country": "Germany"
        },
        "graduation_year": 2009,
        "degree": "PhD",
        "stream": "Computer Science",
        "skills": [
            "Data Analysis",
            "AI",
            "Cloud Computing"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "FinanceCo",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E266",
                "event_name": "Research Conference",
                "date": "2019-07-07",
                "location": "Toronto, Australia"
            },
            {
                "event_id": "E437",
                "event_name": "Annual Alumni Meet",
                "date": "2016-03-18",
                "location": "Boston, Japan"
            }
        ],
        "tags": [
            "Researcher",
            "Finance Specialist",
            "AI Enthusiast"
        ]
    },
    {
        "id": "28",
        "name": "Alice Johnson",
        "email": "user28@example.com",
        "phone": "+19-805-9169",
        "location": {
            "city": "Tokyo",
            "state": "CA",
            "country": "USA"
        },
        "graduation_year": 2006,
        "degree": "PhD",
        "stream": "Computer Science",
        "skills": [
            "Risk Management",
            "Python"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "Alpha Solutions",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E626",
                "event_name": "Tech Symposium",
                "date": "2019-01-21",
                "location": "Tokyo, Canada"
            }
        ],
        "tags": [
            "Mentor",
            "AI Enthusiast",
            "Researcher"
        ]
    },
    {
        "id": "29",
        "name": "Sophia Martinez",
        "email": "user29@example.com",
        "phone": "+40-741-4934",
        "location": {
            "city": "Boston",
            "state": "TX",
            "country": "Japan"
        },
        "graduation_year": 2019,
        "degree": "PhD",
        "stream": "Physics",
        "skills": [
            "Web Development",
            "Risk Management",
            "Finance Analysis",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Alpha Solutions",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E595",
                "event_name": "Finance Workshop",
                "date": "2024-01-26",
                "location": "London, Germany"
            },
            {
                "event_id": "E667",
                "event_name": "Tech Symposium",
                "date": "2022-10-01",
                "location": "Tokyo, Australia"
            }
        ],
        "tags": [
            "Researcher"
        ]
    },
    {
        "id": "30",
        "name": "David Lee",
        "email": "user30@example.com",
        "phone": "+58-560-5389",
        "location": {
            "city": "San Francisco",
            "state": "TX",
            "country": "UK"
        },
        "graduation_year": 2017,
        "degree": "M.Sc",
        "stream": "Finance",
        "skills": [
            "Data Analysis",
            "AI",
            "Cloud Computing",
            "Risk Management",
            "Python"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "TechCorp",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E973",
                "event_name": "Research Conference",
                "date": "2023-11-28",
                "location": "New York, USA"
            },
            {
                "event_id": "E274",
                "event_name": "Research Conference",
                "date": "2018-09-18",
                "location": "London, Canada"
            },
            {
                "event_id": "E332",
                "event_name": "Research Conference",
                "date": "2016-12-07",
                "location": "Boston, Japan"
            }
        ],
        "tags": [
            "AI Enthusiast"
        ]
    },
    {
        "id": "31",
        "name": "Michael Wilson",
        "email": "user31@example.com",
        "phone": "+5-480-7483",
        "location": {
            "city": "Toronto",
            "state": "CA",
            "country": "Canada"
        },
        "graduation_year": 2017,
        "degree": "PhD",
        "stream": "Physics",
        "skills": [
            "Data Analysis",
            "Web Development",
            "Finance Analysis",
            "Python",
            "Risk Management"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Global Analytics",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E314",
                "event_name": "Research Conference",
                "date": "2024-09-10",
                "location": "Tokyo, Canada"
            }
        ],
        "tags": [
            "Finance Specialist"
        ]
    },
    {
        "id": "32",
        "name": "Michael Wilson",
        "email": "user32@example.com",
        "phone": "+78-416-5388",
        "location": {
            "city": "San Francisco",
            "state": "TX",
            "country": "UK"
        },
        "graduation_year": 2020,
        "degree": "PhD",
        "stream": "Finance",
        "skills": [
            "Python",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "FinanceCo",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E134",
                "event_name": "Research Conference",
                "date": "2022-10-19",
                "location": "Berlin, UK"
            },
            {
                "event_id": "E965",
                "event_name": "Annual Alumni Meet",
                "date": "2024-11-09",
                "location": "Sydney, Japan"
            }
        ],
        "tags": [
            "Entrepreneur"
        ]
    },
    {
        "id": "33",
        "name": "Sarah Taylor",
        "email": "user33@example.com",
        "phone": "+17-638-5902",
        "location": {
            "city": "New York",
            "state": "MA",
            "country": "Japan"
        },
        "graduation_year": 2011,
        "degree": "B.Tech",
        "stream": "Physics",
        "skills": [
            "Data Analysis",
            "Machine Learning",
            "AI",
            "Cloud Computing",
            "Finance Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "FinanceCo",
            "location": "New York"
        },
        "events_attended": [
            {
                "event_id": "E905",
                "event_name": "Finance Workshop",
                "date": "2017-02-03",
                "location": "Toronto, Australia"
            },
            {
                "event_id": "E111",
                "event_name": "Tech Symposium",
                "date": "2024-02-26",
                "location": "Sydney, Germany"
            },
            {
                "event_id": "E238",
                "event_name": "Research Conference",
                "date": "2015-06-27",
                "location": "Sydney, UK"
            }
        ],
        "tags": [
            "AI Enthusiast",
            "Mentor",
            "Researcher"
        ]
    },
    {
        "id": "34",
        "name": "Alice Johnson",
        "email": "user34@example.com",
        "phone": "+20-120-5073",
        "location": {
            "city": "Sydney",
            "state": "CA",
            "country": "USA"
        },
        "graduation_year": 2009,
        "degree": "MBA",
        "stream": "Finance",
        "skills": [
            "Risk Management",
            "Web Development",
            "Finance Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "FinanceCo",
            "location": "New York"
        },
        "events_attended": [
            {
                "event_id": "E541",
                "event_name": "Tech Symposium",
                "date": "2018-08-08",
                "location": "New York, Japan"
            }
        ],
        "tags": [
            "Developer",
            "Mentor"
        ]
    },
    {
        "id": "35",
        "name": "Michael Wilson",
        "email": "user35@example.com",
        "phone": "+76-235-1178",
        "location": {
            "city": "Berlin",
            "state": "MA",
            "country": "Australia"
        },
        "graduation_year": 2014,
        "degree": "B.Tech",
        "stream": "Electronics",
        "skills": [
            "Python",
            "Machine Learning",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "FinanceCo",
            "location": "New York"
        },
        "events_attended": [
            {
                "event_id": "E863",
                "event_name": "Finance Workshop",
                "date": "2024-02-15",
                "location": "New York, Australia"
            },
            {
                "event_id": "E792",
                "event_name": "Annual Alumni Meet",
                "date": "2015-04-16",
                "location": "Boston, Germany"
            },
            {
                "event_id": "E559",
                "event_name": "Annual Alumni Meet",
                "date": "2019-11-07",
                "location": "San Francisco, Germany"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Entrepreneur",
            "AI Enthusiast"
        ]
    },
    {
        "id": "36",
        "name": "Alice Johnson",
        "email": "user36@example.com",
        "phone": "+89-247-5301",
        "location": {
            "city": "Toronto",
            "state": "CA",
            "country": "UK"
        },
        "graduation_year": 2018,
        "degree": "M.Sc",
        "stream": "Biology",
        "skills": [
            "Risk Management",
            "Cloud Computing",
            "Machine Learning",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Innovatech",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E140",
                "event_name": "Annual Alumni Meet",
                "date": "2023-03-27",
                "location": "Boston, Canada"
            },
            {
                "event_id": "E752",
                "event_name": "Tech Symposium",
                "date": "2024-07-07",
                "location": "Berlin, Germany"
            },
            {
                "event_id": "E568",
                "event_name": "Annual Alumni Meet",
                "date": "2021-09-03",
                "location": "New York, Germany"
            }
        ],
        "tags": [
            "AI Enthusiast",
            "Mentor"
        ]
    },
    {
        "id": "37",
        "name": "Jane Smith",
        "email": "user37@example.com",
        "phone": "+70-856-7083",
        "location": {
            "city": "Tokyo",
            "state": "",
            "country": "UK"
        },
        "graduation_year": 2013,
        "degree": "PhD",
        "stream": "Mechanical Engineering",
        "skills": [
            "AI",
            "Web Development",
            "Finance Analysis",
            "Cloud Computing",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Global Analytics",
            "location": "New York"
        },
        "events_attended": [
            {
                "event_id": "E674",
                "event_name": "Finance Workshop",
                "date": "2024-11-26",
                "location": "London, USA"
            },
            {
                "event_id": "E812",
                "event_name": "Finance Workshop",
                "date": "2021-09-10",
                "location": "Boston, UK"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Developer",
            "Entrepreneur"
        ]
    },
    {
        "id": "38",
        "name": "John Doe",
        "email": "user38@example.com",
        "phone": "+69-332-4481",
        "location": {
            "city": "Sydney",
            "state": "CA",
            "country": "USA"
        },
        "graduation_year": 2019,
        "degree": "B.Tech",
        "stream": "Computer Science",
        "skills": [
            "Machine Learning",
            "Risk Management",
            "Cloud Computing"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "FinanceCo",
            "location": "Berlin"
        },
        "events_attended": [
            {
                "event_id": "E717",
                "event_name": "Tech Symposium",
                "date": "2022-03-10",
                "location": "San Francisco, UK"
            },
            {
                "event_id": "E825",
                "event_name": "Annual Alumni Meet",
                "date": "2018-11-25",
                "location": "New York, UK"
            },
            {
                "event_id": "E864",
                "event_name": "Annual Alumni Meet",
                "date": "2023-01-14",
                "location": "Sydney, Germany"
            }
        ],
        "tags": [
            "Mentor"
        ]
    },
    {
        "id": "39",
        "name": "Jane Smith",
        "email": "user39@example.com",
        "phone": "+65-550-7035",
        "location": {
            "city": "Tokyo",
            "state": "CA",
            "country": "UK"
        },
        "graduation_year": 2013,
        "degree": "B.Tech",
        "stream": "Physics",
        "skills": [
            "Cloud Computing",
            "Finance Analysis",
            "AI",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Alpha Solutions",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E213",
                "event_name": "Tech Symposium",
                "date": "2024-04-23",
                "location": "San Francisco, Japan"
            },
            {
                "event_id": "E790",
                "event_name": "Annual Alumni Meet",
                "date": "2015-05-22",
                "location": "London, Germany"
            }
        ],
        "tags": [
            "AI Enthusiast",
            "Researcher",
            "Entrepreneur"
        ]
    },
    {
        "id": "40",
        "name": "Robert Brown",
        "email": "user40@example.com",
        "phone": "+8-103-6158",
        "location": {
            "city": "Boston",
            "state": "",
            "country": "Germany"
        },
        "graduation_year": 2023,
        "degree": "B.Tech",
        "stream": "Computer Science",
        "skills": [
            "Risk Management",
            "Web Development",
            "Finance Analysis",
            "AI"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "Alpha Solutions",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E156",
                "event_name": "Annual Alumni Meet",
                "date": "2022-03-17",
                "location": "Berlin, Australia"
            },
            {
                "event_id": "E630",
                "event_name": "Tech Symposium",
                "date": "2021-11-04",
                "location": "Boston, UK"
            },
            {
                "event_id": "E983",
                "event_name": "Annual Alumni Meet",
                "date": "2024-02-02",
                "location": "San Francisco, Germany"
            }
        ],
        "tags": [
            "AI Enthusiast"
        ]
    },
    {
        "id": "41",
        "name": "Sarah Taylor",
        "email": "user41@example.com",
        "phone": "+62-634-3623",
        "location": {
            "city": "Toronto",
            "state": "MA",
            "country": "USA"
        },
        "graduation_year": 2015,
        "degree": "MBA",
        "stream": "Electronics",
        "skills": [
            "Python",
            "Data Analysis",
            "Risk Management"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "FinanceCo",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E763",
                "event_name": "Tech Symposium",
                "date": "2015-08-03",
                "location": "Sydney, UK"
            },
            {
                "event_id": "E561",
                "event_name": "Finance Workshop",
                "date": "2018-07-04",
                "location": "Berlin, USA"
            }
        ],
        "tags": [
            "Developer",
            "Mentor"
        ]
    },
    {
        "id": "42",
        "name": "Michael Wilson",
        "email": "user42@example.com",
        "phone": "+6-698-8519",
        "location": {
            "city": "Tokyo",
            "state": "CA",
            "country": "Australia"
        },
        "graduation_year": 2013,
        "degree": "M.Sc",
        "stream": "Finance",
        "skills": [
            "Data Analysis",
            "Risk Management"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Innovatech",
            "location": "Berlin"
        },
        "events_attended": [
            {
                "event_id": "E402",
                "event_name": "Finance Workshop",
                "date": "2020-02-07",
                "location": "Toronto, Japan"
            },
            {
                "event_id": "E236",
                "event_name": "Annual Alumni Meet",
                "date": "2017-04-20",
                "location": "London, USA"
            }
        ],
        "tags": [
            "Mentor",
            "Finance Specialist"
        ]
    },
    {
        "id": "43",
        "name": "James Anderson",
        "email": "user43@example.com",
        "phone": "+23-317-2785",
        "location": {
            "city": "Toronto",
            "state": "TX",
            "country": "UK"
        },
        "graduation_year": 2018,
        "degree": "B.Tech",
        "stream": "Physics",
        "skills": [
            "Data Analysis",
            "Web Development",
            "Python"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "Innovatech",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E674",
                "event_name": "Research Conference",
                "date": "2020-04-14",
                "location": "Berlin, USA"
            },
            {
                "event_id": "E559",
                "event_name": "Research Conference",
                "date": "2020-06-19",
                "location": "Toronto, USA"
            },
            {
                "event_id": "E186",
                "event_name": "Annual Alumni Meet",
                "date": "2022-12-02",
                "location": "San Francisco, USA"
            }
        ],
        "tags": [
            "AI Enthusiast",
            "Researcher"
        ]
    },
    {
        "id": "44",
        "name": "James Anderson",
        "email": "user44@example.com",
        "phone": "+95-799-8326",
        "location": {
            "city": "San Francisco",
            "state": "TX",
            "country": "USA"
        },
        "graduation_year": 2018,
        "degree": "M.Sc",
        "stream": "Physics",
        "skills": [
            "Cloud Computing",
            "AI",
            "Python",
            "Finance Analysis",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "Global Analytics",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E240",
                "event_name": "Finance Workshop",
                "date": "2015-10-15",
                "location": "New York, UK"
            },
            {
                "event_id": "E149",
                "event_name": "Finance Workshop",
                "date": "2017-01-26",
                "location": "San Francisco, Canada"
            },
            {
                "event_id": "E361",
                "event_name": "Annual Alumni Meet",
                "date": "2019-10-18",
                "location": "Boston, Canada"
            }
        ],
        "tags": [
            "Developer"
        ]
    },
    {
        "id": "45",
        "name": "David Lee",
        "email": "user45@example.com",
        "phone": "+31-935-5985",
        "location": {
            "city": "Sydney",
            "state": "NY",
            "country": "UK"
        },
        "graduation_year": 2020,
        "degree": "M.Sc",
        "stream": "Finance",
        "skills": [
            "Data Analysis",
            "Finance Analysis",
            "Web Development",
            "Risk Management",
            "AI"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Alpha Solutions",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E170",
                "event_name": "Annual Alumni Meet",
                "date": "2016-08-09",
                "location": "Tokyo, Japan"
            },
            {
                "event_id": "E498",
                "event_name": "Research Conference",
                "date": "2024-12-20",
                "location": "New York, Canada"
            },
            {
                "event_id": "E501",
                "event_name": "Annual Alumni Meet",
                "date": "2015-12-21",
                "location": "Berlin, USA"
            }
        ],
        "tags": [
            "Entrepreneur",
            "Mentor"
        ]
    },
    {
        "id": "46",
        "name": "James Anderson",
        "email": "user46@example.com",
        "phone": "+24-946-3740",
        "location": {
            "city": "London",
            "state": "CA",
            "country": "USA"
        },
        "graduation_year": 2010,
        "degree": "B.Tech",
        "stream": "Biology",
        "skills": [
            "Finance Analysis",
            "Cloud Computing",
            "Machine Learning",
            "Risk Management"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "FinanceCo",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E876",
                "event_name": "Research Conference",
                "date": "2021-02-22",
                "location": "Berlin, Australia"
            },
            {
                "event_id": "E240",
                "event_name": "Research Conference",
                "date": "2017-02-23",
                "location": "Berlin, Germany"
            }
        ],
        "tags": [
            "Researcher"
        ]
    },
    {
        "id": "47",
        "name": "John Doe",
        "email": "user47@example.com",
        "phone": "+54-501-7827",
        "location": {
            "city": "New York",
            "state": "NY",
            "country": "Japan"
        },
        "graduation_year": 2011,
        "degree": "MBA",
        "stream": "Biology",
        "skills": [
            "Cloud Computing",
            "Finance Analysis"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "TechCorp",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E839",
                "event_name": "Annual Alumni Meet",
                "date": "2018-07-21",
                "location": "Sydney, Japan"
            },
            {
                "event_id": "E593",
                "event_name": "Finance Workshop",
                "date": "2023-09-21",
                "location": "Sydney, USA"
            },
            {
                "event_id": "E611",
                "event_name": "Annual Alumni Meet",
                "date": "2022-12-28",
                "location": "London, UK"
            }
        ],
        "tags": [
            "Entrepreneur"
        ]
    },
    {
        "id": "48",
        "name": "Michael Wilson",
        "email": "user48@example.com",
        "phone": "+8-312-2704",
        "location": {
            "city": "London",
            "state": "MA",
            "country": "Australia"
        },
        "graduation_year": 2015,
        "degree": "B.Tech",
        "stream": "Physics",
        "skills": [
            "Finance Analysis",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Innovatech",
            "location": "Tokyo"
        },
        "events_attended": [
            {
                "event_id": "E247",
                "event_name": "Research Conference",
                "date": "2015-05-20",
                "location": "New York, Canada"
            }
        ],
        "tags": [
            "Entrepreneur"
        ]
    },
    {
        "id": "49",
        "name": "Alice Johnson",
        "email": "user49@example.com",
        "phone": "+81-822-8863",
        "location": {
            "city": "San Francisco",
            "state": "TX",
            "country": "Australia"
        },
        "graduation_year": 2020,
        "degree": "PhD",
        "stream": "Physics",
        "skills": [
            "Python",
            "Cloud Computing",
            "Web Development",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Alpha Solutions",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E478",
                "event_name": "Tech Symposium",
                "date": "2015-07-11",
                "location": "Berlin, Australia"
            },
            {
                "event_id": "E192",
                "event_name": "Finance Workshop",
                "date": "2018-04-21",
                "location": "Boston, UK"
            }
        ],
        "tags": [
            "Finance Specialist"
        ]
    },
    {
        "id": "50",
        "name": "Sophia Martinez",
        "email": "user50@example.com",
        "phone": "+57-545-4245",
        "location": {
            "city": "London",
            "state": "",
            "country": "Canada"
        },
        "graduation_year": 2002,
        "degree": "M.Sc",
        "stream": "Biology",
        "skills": [
            "Machine Learning",
            "Web Development",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "Global Analytics",
            "location": "New York"
        },
        "events_attended": [
            {
                "event_id": "E153",
                "event_name": "Finance Workshop",
                "date": "2024-12-15",
                "location": "London, USA"
            },
            {
                "event_id": "E551",
                "event_name": "Research Conference",
                "date": "2016-11-18",
                "location": "San Francisco, USA"
            },
            {
                "event_id": "E369",
                "event_name": "Tech Symposium",
                "date": "2021-06-15",
                "location": "San Francisco, USA"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Entrepreneur"
        ]
    },
    {
        "id": "51",
        "name": "Michael Wilson",
        "email": "user51@example.com",
        "phone": "+4-689-7290",
        "location": {
            "city": "Sydney",
            "state": "MA",
            "country": "UK"
        },
        "graduation_year": 2023,
        "degree": "MBA",
        "stream": "Mechanical Engineering",
        "skills": [
            "Web Development",
            "Python",
            "Machine Learning",
            "AI",
            "Cloud Computing"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "FinanceCo",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E690",
                "event_name": "Tech Symposium",
                "date": "2018-01-19",
                "location": "London, Germany"
            },
            {
                "event_id": "E522",
                "event_name": "Finance Workshop",
                "date": "2017-10-23",
                "location": "Boston, Australia"
            },
            {
                "event_id": "E861",
                "event_name": "Tech Symposium",
                "date": "2020-03-09",
                "location": "Berlin, Canada"
            }
        ],
        "tags": [
            "Researcher",
            "Mentor"
        ]
    },
    {
        "id": "52",
        "name": "Emily Davis",
        "email": "user52@example.com",
        "phone": "+51-419-5815",
        "location": {
            "city": "Sydney",
            "state": "",
            "country": "USA"
        },
        "graduation_year": 2000,
        "degree": "PhD",
        "stream": "Biology",
        "skills": [
            "Data Analysis",
            "Risk Management",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "FinanceCo",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E916",
                "event_name": "Research Conference",
                "date": "2018-07-13",
                "location": "Toronto, USA"
            },
            {
                "event_id": "E788",
                "event_name": "Finance Workshop",
                "date": "2017-08-20",
                "location": "San Francisco, Canada"
            },
            {
                "event_id": "E375",
                "event_name": "Tech Symposium",
                "date": "2024-06-11",
                "location": "Tokyo, USA"
            }
        ],
        "tags": [
            "AI Enthusiast"
        ]
    },
    {
        "id": "53",
        "name": "John Doe",
        "email": "user53@example.com",
        "phone": "+47-906-9629",
        "location": {
            "city": "New York",
            "state": "",
            "country": "Australia"
        },
        "graduation_year": 2023,
        "degree": "B.Tech",
        "stream": "Physics",
        "skills": [
            "AI",
            "Finance Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "FinanceCo",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E416",
                "event_name": "Research Conference",
                "date": "2020-09-23",
                "location": "San Francisco, Canada"
            }
        ],
        "tags": [
            "Entrepreneur",
            "Researcher",
            "Finance Specialist"
        ]
    },
    {
        "id": "54",
        "name": "Emily Davis",
        "email": "user54@example.com",
        "phone": "+54-553-3148",
        "location": {
            "city": "London",
            "state": "",
            "country": "UK"
        },
        "graduation_year": 2013,
        "degree": "PhD",
        "stream": "Finance",
        "skills": [
            "Python",
            "Cloud Computing",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "FinanceCo",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E821",
                "event_name": "Research Conference",
                "date": "2019-10-19",
                "location": "New York, Germany"
            },
            {
                "event_id": "E946",
                "event_name": "Tech Symposium",
                "date": "2022-03-03",
                "location": "San Francisco, UK"
            },
            {
                "event_id": "E479",
                "event_name": "Research Conference",
                "date": "2021-04-17",
                "location": "London, Australia"
            }
        ],
        "tags": [
            "Entrepreneur",
            "Developer",
            "AI Enthusiast"
        ]
    },
    {
        "id": "55",
        "name": "Alice Johnson",
        "email": "user55@example.com",
        "phone": "+22-372-3195",
        "location": {
            "city": "London",
            "state": "",
            "country": "Canada"
        },
        "graduation_year": 2014,
        "degree": "M.Sc",
        "stream": "Biology",
        "skills": [
            "Finance Analysis",
            "Python"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Innovatech",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E339",
                "event_name": "Finance Workshop",
                "date": "2018-03-07",
                "location": "New York, Canada"
            },
            {
                "event_id": "E234",
                "event_name": "Annual Alumni Meet",
                "date": "2023-09-28",
                "location": "London, Germany"
            }
        ],
        "tags": [
            "Mentor"
        ]
    },
    {
        "id": "56",
        "name": "Jane Smith",
        "email": "user56@example.com",
        "phone": "+54-773-9764",
        "location": {
            "city": "New York",
            "state": "NY",
            "country": "Japan"
        },
        "graduation_year": 2018,
        "degree": "PhD",
        "stream": "Finance",
        "skills": [
            "AI",
            "Machine Learning",
            "Risk Management",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "FinanceCo",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E772",
                "event_name": "Research Conference",
                "date": "2015-06-13",
                "location": "Tokyo, Australia"
            }
        ],
        "tags": [
            "Researcher",
            "Entrepreneur",
            "Mentor"
        ]
    },
    {
        "id": "57",
        "name": "Sarah Taylor",
        "email": "user57@example.com",
        "phone": "+70-253-9651",
        "location": {
            "city": "Tokyo",
            "state": "NY",
            "country": "Australia"
        },
        "graduation_year": 2011,
        "degree": "MBA",
        "stream": "Biology",
        "skills": [
            "Web Development",
            "Data Analysis",
            "Python",
            "Cloud Computing",
            "AI"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "TechCorp",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E491",
                "event_name": "Research Conference",
                "date": "2019-08-24",
                "location": "San Francisco, Australia"
            },
            {
                "event_id": "E214",
                "event_name": "Finance Workshop",
                "date": "2021-01-10",
                "location": "Sydney, UK"
            },
            {
                "event_id": "E762",
                "event_name": "Tech Symposium",
                "date": "2020-02-04",
                "location": "Toronto, Germany"
            }
        ],
        "tags": [
            "Mentor",
            "Developer"
        ]
    },
    {
        "id": "58",
        "name": "Emily Davis",
        "email": "user58@example.com",
        "phone": "+14-235-9001",
        "location": {
            "city": "New York",
            "state": "TX",
            "country": "Canada"
        },
        "graduation_year": 2007,
        "degree": "MBA",
        "stream": "Biology",
        "skills": [
            "Finance Analysis",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Global Analytics",
            "location": "New York"
        },
        "events_attended": [
            {
                "event_id": "E885",
                "event_name": "Finance Workshop",
                "date": "2022-08-16",
                "location": "San Francisco, USA"
            }
        ],
        "tags": [
            "Researcher"
        ]
    },
    {
        "id": "59",
        "name": "Alice Johnson",
        "email": "user59@example.com",
        "phone": "+70-635-3522",
        "location": {
            "city": "Boston",
            "state": "TX",
            "country": "Australia"
        },
        "graduation_year": 2014,
        "degree": "PhD",
        "stream": "Computer Science",
        "skills": [
            "Finance Analysis",
            "Python"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "FinanceCo",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E224",
                "event_name": "Annual Alumni Meet",
                "date": "2018-02-28",
                "location": "London, Germany"
            },
            {
                "event_id": "E766",
                "event_name": "Annual Alumni Meet",
                "date": "2017-01-03",
                "location": "Toronto, UK"
            },
            {
                "event_id": "E455",
                "event_name": "Research Conference",
                "date": "2019-09-27",
                "location": "Sydney, Germany"
            }
        ],
        "tags": [
            "Developer",
            "Mentor"
        ]
    },
    {
        "id": "60",
        "name": "Emily Davis",
        "email": "user60@example.com",
        "phone": "+40-642-2828",
        "location": {
            "city": "London",
            "state": "CA",
            "country": "Canada"
        },
        "graduation_year": 2006,
        "degree": "M.Sc",
        "stream": "Computer Science",
        "skills": [
            "AI",
            "Risk Management",
            "Finance Analysis"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "FinanceCo",
            "location": "New York"
        },
        "events_attended": [
            {
                "event_id": "E115",
                "event_name": "Tech Symposium",
                "date": "2024-11-09",
                "location": "Toronto, UK"
            }
        ],
        "tags": [
            "Mentor"
        ]
    },
    {
        "id": "61",
        "name": "Jane Smith",
        "email": "user61@example.com",
        "phone": "+11-522-4797",
        "location": {
            "city": "Boston",
            "state": "CA",
            "country": "Japan"
        },
        "graduation_year": 2000,
        "degree": "B.Tech",
        "stream": "Mechanical Engineering",
        "skills": [
            "AI",
            "Risk Management",
            "Cloud Computing",
            "Web Development"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Alpha Solutions",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E774",
                "event_name": "Tech Symposium",
                "date": "2018-07-04",
                "location": "Sydney, Japan"
            },
            {
                "event_id": "E414",
                "event_name": "Finance Workshop",
                "date": "2022-03-07",
                "location": "New York, Canada"
            },
            {
                "event_id": "E382",
                "event_name": "Tech Symposium",
                "date": "2016-09-11",
                "location": "Toronto, Australia"
            }
        ],
        "tags": [
            "Developer"
        ]
    },
    {
        "id": "62",
        "name": "James Anderson",
        "email": "user62@example.com",
        "phone": "+41-164-2395",
        "location": {
            "city": "San Francisco",
            "state": "CA",
            "country": "Germany"
        },
        "graduation_year": 2018,
        "degree": "B.Tech",
        "stream": "Computer Science",
        "skills": [
            "Web Development",
            "Finance Analysis"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "FinanceCo",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E446",
                "event_name": "Annual Alumni Meet",
                "date": "2022-07-01",
                "location": "Boston, USA"
            },
            {
                "event_id": "E747",
                "event_name": "Annual Alumni Meet",
                "date": "2016-12-21",
                "location": "Sydney, Canada"
            },
            {
                "event_id": "E722",
                "event_name": "Finance Workshop",
                "date": "2020-06-16",
                "location": "Boston, Canada"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Researcher",
            "AI Enthusiast"
        ]
    },
    {
        "id": "63",
        "name": "James Anderson",
        "email": "user63@example.com",
        "phone": "+65-440-2509",
        "location": {
            "city": "Tokyo",
            "state": "TX",
            "country": "Australia"
        },
        "graduation_year": 2019,
        "degree": "M.Sc",
        "stream": "Biology",
        "skills": [
            "Cloud Computing",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "FinanceCo",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E982",
                "event_name": "Finance Workshop",
                "date": "2015-09-02",
                "location": "Boston, Japan"
            },
            {
                "event_id": "E322",
                "event_name": "Annual Alumni Meet",
                "date": "2021-12-25",
                "location": "Toronto, USA"
            }
        ],
        "tags": [
            "Mentor"
        ]
    },
    {
        "id": "64",
        "name": "Sarah Taylor",
        "email": "user64@example.com",
        "phone": "+21-441-3527",
        "location": {
            "city": "Toronto",
            "state": "MA",
            "country": "UK"
        },
        "graduation_year": 2004,
        "degree": "B.Tech",
        "stream": "Electronics",
        "skills": [
            "Finance Analysis",
            "AI",
            "Risk Management",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "FinanceCo",
            "location": "Berlin"
        },
        "events_attended": [
            {
                "event_id": "E807",
                "event_name": "Tech Symposium",
                "date": "2024-01-28",
                "location": "New York, Canada"
            },
            {
                "event_id": "E435",
                "event_name": "Tech Symposium",
                "date": "2016-11-27",
                "location": "Boston, Germany"
            },
            {
                "event_id": "E673",
                "event_name": "Tech Symposium",
                "date": "2023-04-10",
                "location": "Tokyo, USA"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Entrepreneur",
            "AI Enthusiast"
        ]
    },
    {
        "id": "65",
        "name": "David Lee",
        "email": "user65@example.com",
        "phone": "+70-419-7638",
        "location": {
            "city": "London",
            "state": "TX",
            "country": "Japan"
        },
        "graduation_year": 2006,
        "degree": "PhD",
        "stream": "Biology",
        "skills": [
            "Cloud Computing",
            "Machine Learning",
            "AI"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "Innovatech",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E353",
                "event_name": "Finance Workshop",
                "date": "2021-03-14",
                "location": "Berlin, Japan"
            }
        ],
        "tags": [
            "Entrepreneur"
        ]
    },
    {
        "id": "66",
        "name": "Emily Davis",
        "email": "user66@example.com",
        "phone": "+79-997-4880",
        "location": {
            "city": "San Francisco",
            "state": "",
            "country": "Germany"
        },
        "graduation_year": 2003,
        "degree": "M.Sc",
        "stream": "Physics",
        "skills": [
            "Cloud Computing",
            "Risk Management",
            "Web Development",
            "Python",
            "AI"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Alpha Solutions",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E760",
                "event_name": "Tech Symposium",
                "date": "2022-12-08",
                "location": "Berlin, Canada"
            },
            {
                "event_id": "E509",
                "event_name": "Annual Alumni Meet",
                "date": "2022-03-16",
                "location": "Boston, Japan"
            },
            {
                "event_id": "E211",
                "event_name": "Research Conference",
                "date": "2022-04-04",
                "location": "San Francisco, USA"
            }
        ],
        "tags": [
            "AI Enthusiast",
            "Mentor"
        ]
    },
    {
        "id": "67",
        "name": "Alice Johnson",
        "email": "user67@example.com",
        "phone": "+32-217-8492",
        "location": {
            "city": "London",
            "state": "TX",
            "country": "UK"
        },
        "graduation_year": 2007,
        "degree": "M.Sc",
        "stream": "Computer Science",
        "skills": [
            "Risk Management",
            "Data Analysis",
            "Python"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "FinanceCo",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E536",
                "event_name": "Finance Workshop",
                "date": "2018-01-25",
                "location": "Tokyo, UK"
            },
            {
                "event_id": "E971",
                "event_name": "Research Conference",
                "date": "2016-09-28",
                "location": "Boston, UK"
            }
        ],
        "tags": [
            "Mentor"
        ]
    },
    {
        "id": "68",
        "name": "John Doe",
        "email": "user68@example.com",
        "phone": "+32-580-7708",
        "location": {
            "city": "San Francisco",
            "state": "TX",
            "country": "USA"
        },
        "graduation_year": 2017,
        "degree": "M.Sc",
        "stream": "Biology",
        "skills": [
            "AI",
            "Risk Management",
            "Finance Analysis",
            "Web Development",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "FinanceCo",
            "location": "Berlin"
        },
        "events_attended": [
            {
                "event_id": "E575",
                "event_name": "Research Conference",
                "date": "2021-10-08",
                "location": "Tokyo, Australia"
            }
        ],
        "tags": [
            "Developer"
        ]
    },
    {
        "id": "69",
        "name": "David Lee",
        "email": "user69@example.com",
        "phone": "+85-294-6681",
        "location": {
            "city": "New York",
            "state": "MA",
            "country": "UK"
        },
        "graduation_year": 2020,
        "degree": "PhD",
        "stream": "Finance",
        "skills": [
            "Python",
            "Cloud Computing",
            "Risk Management",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "TechCorp",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E546",
                "event_name": "Annual Alumni Meet",
                "date": "2016-11-12",
                "location": "San Francisco, UK"
            }
        ],
        "tags": [
            "Mentor",
            "Finance Specialist"
        ]
    },
    {
        "id": "70",
        "name": "David Lee",
        "email": "user70@example.com",
        "phone": "+55-328-9754",
        "location": {
            "city": "Toronto",
            "state": "MA",
            "country": "UK"
        },
        "graduation_year": 2008,
        "degree": "M.Sc",
        "stream": "Mechanical Engineering",
        "skills": [
            "Cloud Computing",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "FinanceCo",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E664",
                "event_name": "Annual Alumni Meet",
                "date": "2023-06-25",
                "location": "New York, Germany"
            }
        ],
        "tags": [
            "Mentor",
            "AI Enthusiast"
        ]
    },
    {
        "id": "71",
        "name": "James Anderson",
        "email": "user71@example.com",
        "phone": "+69-238-7764",
        "location": {
            "city": "London",
            "state": "CA",
            "country": "Canada"
        },
        "graduation_year": 2018,
        "degree": "B.Tech",
        "stream": "Finance",
        "skills": [
            "AI",
            "Python",
            "Web Development",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "Alpha Solutions",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E903",
                "event_name": "Annual Alumni Meet",
                "date": "2020-12-08",
                "location": "Berlin, Japan"
            },
            {
                "event_id": "E402",
                "event_name": "Research Conference",
                "date": "2019-11-10",
                "location": "New York, Japan"
            }
        ],
        "tags": [
            "Developer",
            "Entrepreneur"
        ]
    },
    {
        "id": "72",
        "name": "John Doe",
        "email": "user72@example.com",
        "phone": "+80-275-2446",
        "location": {
            "city": "Toronto",
            "state": "",
            "country": "Canada"
        },
        "graduation_year": 2022,
        "degree": "MBA",
        "stream": "Computer Science",
        "skills": [
            "Data Analysis",
            "Risk Management",
            "Python",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "Global Analytics",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E469",
                "event_name": "Research Conference",
                "date": "2022-11-03",
                "location": "Toronto, Germany"
            },
            {
                "event_id": "E292",
                "event_name": "Research Conference",
                "date": "2020-09-11",
                "location": "New York, Canada"
            }
        ],
        "tags": [
            "Finance Specialist"
        ]
    },
    {
        "id": "73",
        "name": "James Anderson",
        "email": "user73@example.com",
        "phone": "+51-766-9357",
        "location": {
            "city": "Tokyo",
            "state": "MA",
            "country": "UK"
        },
        "graduation_year": 2013,
        "degree": "MBA",
        "stream": "Finance",
        "skills": [
            "Web Development",
            "AI"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "TechCorp",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E953",
                "event_name": "Tech Symposium",
                "date": "2024-02-26",
                "location": "Boston, USA"
            }
        ],
        "tags": [
            "Developer",
            "Mentor",
            "Researcher"
        ]
    },
    {
        "id": "74",
        "name": "Sarah Taylor",
        "email": "user74@example.com",
        "phone": "+54-333-7025",
        "location": {
            "city": "Toronto",
            "state": "MA",
            "country": "Germany"
        },
        "graduation_year": 2008,
        "degree": "B.Tech",
        "stream": "Physics",
        "skills": [
            "Data Analysis",
            "AI",
            "Python"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "FinanceCo",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E110",
                "event_name": "Finance Workshop",
                "date": "2022-04-07",
                "location": "Tokyo, Germany"
            },
            {
                "event_id": "E536",
                "event_name": "Research Conference",
                "date": "2021-05-19",
                "location": "Toronto, Canada"
            }
        ],
        "tags": [
            "Entrepreneur",
            "Developer",
            "AI Enthusiast"
        ]
    },
    {
        "id": "75",
        "name": "Emily Davis",
        "email": "user75@example.com",
        "phone": "+93-715-3054",
        "location": {
            "city": "Berlin",
            "state": "CA",
            "country": "Canada"
        },
        "graduation_year": 2008,
        "degree": "B.Tech",
        "stream": "Electronics",
        "skills": [
            "Risk Management",
            "Cloud Computing",
            "Python",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Innovatech",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E922",
                "event_name": "Tech Symposium",
                "date": "2022-04-19",
                "location": "London, USA"
            }
        ],
        "tags": [
            "Developer",
            "AI Enthusiast"
        ]
    },
    {
        "id": "76",
        "name": "Sophia Martinez",
        "email": "user76@example.com",
        "phone": "+29-215-8961",
        "location": {
            "city": "London",
            "state": "MA",
            "country": "Australia"
        },
        "graduation_year": 2006,
        "degree": "M.Sc",
        "stream": "Physics",
        "skills": [
            "AI",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "FinanceCo",
            "location": "Berlin"
        },
        "events_attended": [
            {
                "event_id": "E892",
                "event_name": "Finance Workshop",
                "date": "2024-06-03",
                "location": "New York, Canada"
            },
            {
                "event_id": "E578",
                "event_name": "Tech Symposium",
                "date": "2024-02-21",
                "location": "Tokyo, Germany"
            }
        ],
        "tags": [
            "Developer"
        ]
    },
    {
        "id": "77",
        "name": "Sarah Taylor",
        "email": "user77@example.com",
        "phone": "+39-406-5240",
        "location": {
            "city": "Toronto",
            "state": "CA",
            "country": "Germany"
        },
        "graduation_year": 2020,
        "degree": "M.Sc",
        "stream": "Physics",
        "skills": [
            "Web Development",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Alpha Solutions",
            "location": "Tokyo"
        },
        "events_attended": [
            {
                "event_id": "E493",
                "event_name": "Tech Symposium",
                "date": "2017-07-16",
                "location": "London, UK"
            },
            {
                "event_id": "E167",
                "event_name": "Finance Workshop",
                "date": "2024-06-22",
                "location": "London, Germany"
            },
            {
                "event_id": "E617",
                "event_name": "Tech Symposium",
                "date": "2022-06-10",
                "location": "Sydney, Canada"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Researcher",
            "Mentor"
        ]
    },
    {
        "id": "78",
        "name": "Michael Wilson",
        "email": "user78@example.com",
        "phone": "+56-510-5595",
        "location": {
            "city": "Boston",
            "state": "NY",
            "country": "USA"
        },
        "graduation_year": 2007,
        "degree": "PhD",
        "stream": "Mechanical Engineering",
        "skills": [
            "Machine Learning",
            "Finance Analysis",
            "Python",
            "Cloud Computing"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "FinanceCo",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E501",
                "event_name": "Annual Alumni Meet",
                "date": "2021-12-06",
                "location": "Berlin, Japan"
            },
            {
                "event_id": "E396",
                "event_name": "Research Conference",
                "date": "2015-02-22",
                "location": "Toronto, USA"
            }
        ],
        "tags": [
            "AI Enthusiast"
        ]
    },
    {
        "id": "79",
        "name": "James Anderson",
        "email": "user79@example.com",
        "phone": "+13-507-9799",
        "location": {
            "city": "New York",
            "state": "NY",
            "country": "Australia"
        },
        "graduation_year": 2023,
        "degree": "PhD",
        "stream": "Biology",
        "skills": [
            "AI",
            "Cloud Computing",
            "Web Development",
            "Finance Analysis",
            "Risk Management"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "FinanceCo",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E298",
                "event_name": "Finance Workshop",
                "date": "2023-05-03",
                "location": "San Francisco, Canada"
            },
            {
                "event_id": "E602",
                "event_name": "Annual Alumni Meet",
                "date": "2017-07-27",
                "location": "New York, Japan"
            }
        ],
        "tags": [
            "Developer",
            "Entrepreneur"
        ]
    },
    {
        "id": "80",
        "name": "Emily Davis",
        "email": "user80@example.com",
        "phone": "+28-230-5380",
        "location": {
            "city": "Sydney",
            "state": "MA",
            "country": "Australia"
        },
        "graduation_year": 2006,
        "degree": "B.Tech",
        "stream": "Biology",
        "skills": [
            "Python",
            "Cloud Computing",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Alpha Solutions",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E677",
                "event_name": "Tech Symposium",
                "date": "2015-07-01",
                "location": "Toronto, Canada"
            }
        ],
        "tags": [
            "Researcher",
            "Entrepreneur",
            "Mentor"
        ]
    },
    {
        "id": "81",
        "name": "Sophia Martinez",
        "email": "user81@example.com",
        "phone": "+1-444-8216",
        "location": {
            "city": "Toronto",
            "state": "",
            "country": "Germany"
        },
        "graduation_year": 2019,
        "degree": "PhD",
        "stream": "Physics",
        "skills": [
            "AI",
            "Web Development",
            "Risk Management"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Global Analytics",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E155",
                "event_name": "Finance Workshop",
                "date": "2020-02-27",
                "location": "New York, Japan"
            },
            {
                "event_id": "E284",
                "event_name": "Tech Symposium",
                "date": "2021-04-04",
                "location": "Berlin, USA"
            },
            {
                "event_id": "E808",
                "event_name": "Research Conference",
                "date": "2024-06-10",
                "location": "London, Australia"
            }
        ],
        "tags": [
            "Researcher",
            "Finance Specialist",
            "Developer"
        ]
    },
    {
        "id": "82",
        "name": "Alice Johnson",
        "email": "user82@example.com",
        "phone": "+63-581-3625",
        "location": {
            "city": "San Francisco",
            "state": "",
            "country": "USA"
        },
        "graduation_year": 2019,
        "degree": "MBA",
        "stream": "Physics",
        "skills": [
            "Cloud Computing",
            "Finance Analysis"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "Innovatech",
            "location": "Berlin"
        },
        "events_attended": [
            {
                "event_id": "E101",
                "event_name": "Finance Workshop",
                "date": "2021-09-24",
                "location": "Berlin, Germany"
            },
            {
                "event_id": "E824",
                "event_name": "Annual Alumni Meet",
                "date": "2020-09-13",
                "location": "Toronto, Germany"
            }
        ],
        "tags": [
            "AI Enthusiast",
            "Finance Specialist",
            "Researcher"
        ]
    },
    {
        "id": "83",
        "name": "Jane Smith",
        "email": "user83@example.com",
        "phone": "+29-659-9749",
        "location": {
            "city": "Sydney",
            "state": "",
            "country": "Australia"
        },
        "graduation_year": 2023,
        "degree": "M.Sc",
        "stream": "Finance",
        "skills": [
            "Python",
            "Finance Analysis",
            "Cloud Computing",
            "Risk Management",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Alpha Solutions",
            "location": "Berlin"
        },
        "events_attended": [
            {
                "event_id": "E501",
                "event_name": "Tech Symposium",
                "date": "2017-06-09",
                "location": "Berlin, USA"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Entrepreneur",
            "Researcher"
        ]
    },
    {
        "id": "84",
        "name": "James Anderson",
        "email": "user84@example.com",
        "phone": "+82-894-7981",
        "location": {
            "city": "London",
            "state": "MA",
            "country": "Japan"
        },
        "graduation_year": 2005,
        "degree": "MBA",
        "stream": "Finance",
        "skills": [
            "Data Analysis",
            "Python",
            "Web Development",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "Innovatech",
            "location": "Berlin"
        },
        "events_attended": [
            {
                "event_id": "E625",
                "event_name": "Tech Symposium",
                "date": "2016-01-21",
                "location": "San Francisco, Germany"
            },
            {
                "event_id": "E441",
                "event_name": "Tech Symposium",
                "date": "2022-03-07",
                "location": "Berlin, UK"
            },
            {
                "event_id": "E885",
                "event_name": "Research Conference",
                "date": "2015-05-04",
                "location": "Berlin, Australia"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Developer",
            "Mentor"
        ]
    },
    {
        "id": "85",
        "name": "Jane Smith",
        "email": "user85@example.com",
        "phone": "+55-894-6563",
        "location": {
            "city": "Sydney",
            "state": "NY",
            "country": "Germany"
        },
        "graduation_year": 2002,
        "degree": "M.Sc",
        "stream": "Finance",
        "skills": [
            "AI",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "FinanceCo",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E460",
                "event_name": "Research Conference",
                "date": "2016-03-26",
                "location": "London, Japan"
            },
            {
                "event_id": "E970",
                "event_name": "Annual Alumni Meet",
                "date": "2022-08-26",
                "location": "San Francisco, USA"
            },
            {
                "event_id": "E110",
                "event_name": "Tech Symposium",
                "date": "2021-07-11",
                "location": "Toronto, UK"
            }
        ],
        "tags": [
            "Mentor",
            "Developer"
        ]
    },
    {
        "id": "86",
        "name": "James Anderson",
        "email": "user86@example.com",
        "phone": "+18-696-2693",
        "location": {
            "city": "Toronto",
            "state": "NY",
            "country": "Australia"
        },
        "graduation_year": 2020,
        "degree": "MBA",
        "stream": "Physics",
        "skills": [
            "AI",
            "Cloud Computing",
            "Risk Management",
            "Finance Analysis",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "Alpha Solutions",
            "location": "San Francisco"
        },
        "events_attended": [
            {
                "event_id": "E671",
                "event_name": "Research Conference",
                "date": "2015-10-06",
                "location": "Boston, USA"
            },
            {
                "event_id": "E828",
                "event_name": "Research Conference",
                "date": "2015-10-04",
                "location": "London, Australia"
            }
        ],
        "tags": [
            "Researcher",
            "AI Enthusiast",
            "Entrepreneur"
        ]
    },
    {
        "id": "87",
        "name": "James Anderson",
        "email": "user87@example.com",
        "phone": "+17-868-1069",
        "location": {
            "city": "Berlin",
            "state": "",
            "country": "Canada"
        },
        "graduation_year": 2006,
        "degree": "MBA",
        "stream": "Biology",
        "skills": [
            "Risk Management",
            "Cloud Computing",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "FinanceCo",
            "location": "Boston"
        },
        "events_attended": [
            {
                "event_id": "E854",
                "event_name": "Finance Workshop",
                "date": "2022-04-18",
                "location": "Berlin, UK"
            },
            {
                "event_id": "E394",
                "event_name": "Tech Symposium",
                "date": "2020-12-16",
                "location": "Berlin, Canada"
            }
        ],
        "tags": [
            "AI Enthusiast"
        ]
    },
    {
        "id": "88",
        "name": "David Lee",
        "email": "user88@example.com",
        "phone": "+12-356-3612",
        "location": {
            "city": "Tokyo",
            "state": "CA",
            "country": "Canada"
        },
        "graduation_year": 2004,
        "degree": "MBA",
        "stream": "Computer Science",
        "skills": [
            "Web Development",
            "Finance Analysis",
            "Python"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Global Analytics",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E266",
                "event_name": "Research Conference",
                "date": "2017-03-04",
                "location": "Boston, USA"
            },
            {
                "event_id": "E237",
                "event_name": "Tech Symposium",
                "date": "2024-10-05",
                "location": "Berlin, Canada"
            },
            {
                "event_id": "E500",
                "event_name": "Tech Symposium",
                "date": "2020-09-19",
                "location": "Boston, USA"
            }
        ],
        "tags": [
            "Developer"
        ]
    },
    {
        "id": "89",
        "name": "Sophia Martinez",
        "email": "user89@example.com",
        "phone": "+80-537-4755",
        "location": {
            "city": "Boston",
            "state": "CA",
            "country": "USA"
        },
        "graduation_year": 2001,
        "degree": "MBA",
        "stream": "Biology",
        "skills": [
            "Machine Learning",
            "Finance Analysis",
            "AI",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Innovatech",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E157",
                "event_name": "Research Conference",
                "date": "2022-09-11",
                "location": "New York, USA"
            },
            {
                "event_id": "E803",
                "event_name": "Tech Symposium",
                "date": "2019-08-23",
                "location": "Toronto, Australia"
            }
        ],
        "tags": [
            "Developer"
        ]
    },
    {
        "id": "90",
        "name": "Robert Brown",
        "email": "user90@example.com",
        "phone": "+36-927-7543",
        "location": {
            "city": "Boston",
            "state": "CA",
            "country": "Canada"
        },
        "graduation_year": 2009,
        "degree": "B.Tech",
        "stream": "Physics",
        "skills": [
            "Finance Analysis",
            "Cloud Computing"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Alpha Solutions",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E673",
                "event_name": "Annual Alumni Meet",
                "date": "2018-07-26",
                "location": "San Francisco, Canada"
            }
        ],
        "tags": [
            "Developer",
            "Finance Specialist"
        ]
    },
    {
        "id": "91",
        "name": "Robert Brown",
        "email": "user91@example.com",
        "phone": "+40-522-3869",
        "location": {
            "city": "London",
            "state": "",
            "country": "USA"
        },
        "graduation_year": 2018,
        "degree": "M.Sc",
        "stream": "Computer Science",
        "skills": [
            "Finance Analysis",
            "Machine Learning",
            "AI",
            "Risk Management"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "FinanceCo",
            "location": "Tokyo"
        },
        "events_attended": [
            {
                "event_id": "E632",
                "event_name": "Finance Workshop",
                "date": "2017-12-10",
                "location": "San Francisco, Australia"
            },
            {
                "event_id": "E277",
                "event_name": "Research Conference",
                "date": "2017-10-23",
                "location": "Boston, USA"
            },
            {
                "event_id": "E200",
                "event_name": "Research Conference",
                "date": "2022-04-14",
                "location": "Sydney, Germany"
            }
        ],
        "tags": [
            "Mentor"
        ]
    },
    {
        "id": "92",
        "name": "John Doe",
        "email": "user92@example.com",
        "phone": "+71-688-3216",
        "location": {
            "city": "Berlin",
            "state": "TX",
            "country": "Canada"
        },
        "graduation_year": 2022,
        "degree": "M.Sc",
        "stream": "Biology",
        "skills": [
            "Risk Management",
            "Data Analysis",
            "Finance Analysis",
            "Cloud Computing"
        ],
        "current_employment": {
            "designation": "Researcher",
            "company": "Alpha Solutions",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E219",
                "event_name": "Annual Alumni Meet",
                "date": "2016-04-22",
                "location": "San Francisco, Germany"
            },
            {
                "event_id": "E737",
                "event_name": "Research Conference",
                "date": "2015-07-03",
                "location": "San Francisco, USA"
            }
        ],
        "tags": [
            "Finance Specialist",
            "Researcher"
        ]
    },
    {
        "id": "93",
        "name": "Michael Wilson",
        "email": "user93@example.com",
        "phone": "+32-165-2622",
        "location": {
            "city": "Boston",
            "state": "CA",
            "country": "Australia"
        },
        "graduation_year": 2015,
        "degree": "PhD",
        "stream": "Physics",
        "skills": [
            "Machine Learning",
            "Finance Analysis",
            "AI",
            "Python"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Innovatech",
            "location": "New York"
        },
        "events_attended": [
            {
                "event_id": "E917",
                "event_name": "Research Conference",
                "date": "2022-05-27",
                "location": "London, Japan"
            },
            {
                "event_id": "E662",
                "event_name": "Tech Symposium",
                "date": "2017-10-24",
                "location": "Berlin, Australia"
            },
            {
                "event_id": "E777",
                "event_name": "Finance Workshop",
                "date": "2021-04-13",
                "location": "San Francisco, Germany"
            }
        ],
        "tags": [
            "Entrepreneur",
            "Researcher",
            "Mentor"
        ]
    },
    {
        "id": "94",
        "name": "Emily Davis",
        "email": "user94@example.com",
        "phone": "+91-501-4749",
        "location": {
            "city": "New York",
            "state": "CA",
            "country": "Germany"
        },
        "graduation_year": 2010,
        "degree": "MBA",
        "stream": "Finance",
        "skills": [
            "Data Analysis",
            "Machine Learning"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "Alpha Solutions",
            "location": "Tokyo"
        },
        "events_attended": [
            {
                "event_id": "E438",
                "event_name": "Finance Workshop",
                "date": "2017-12-24",
                "location": "Toronto, Japan"
            },
            {
                "event_id": "E930",
                "event_name": "Annual Alumni Meet",
                "date": "2021-10-02",
                "location": "San Francisco, UK"
            },
            {
                "event_id": "E688",
                "event_name": "Research Conference",
                "date": "2019-03-28",
                "location": "San Francisco, Australia"
            }
        ],
        "tags": [
            "Mentor",
            "Developer",
            "Entrepreneur"
        ]
    },
    {
        "id": "95",
        "name": "Sarah Taylor",
        "email": "user95@example.com",
        "phone": "+61-426-1215",
        "location": {
            "city": "Boston",
            "state": "MA",
            "country": "Germany"
        },
        "graduation_year": 2010,
        "degree": "PhD",
        "stream": "Mechanical Engineering",
        "skills": [
            "Web Development",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "FinanceCo",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E807",
                "event_name": "Tech Symposium",
                "date": "2021-09-12",
                "location": "Toronto, USA"
            },
            {
                "event_id": "E243",
                "event_name": "Tech Symposium",
                "date": "2019-03-25",
                "location": "San Francisco, Canada"
            }
        ],
        "tags": [
            "Entrepreneur",
            "Finance Specialist"
        ]
    },
    {
        "id": "96",
        "name": "James Anderson",
        "email": "user96@example.com",
        "phone": "+62-539-3709",
        "location": {
            "city": "Tokyo",
            "state": "MA",
            "country": "Australia"
        },
        "graduation_year": 2020,
        "degree": "MBA",
        "stream": "Physics",
        "skills": [
            "Cloud Computing",
            "AI",
            "Python",
            "Finance Analysis"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "Global Analytics",
            "location": "Toronto"
        },
        "events_attended": [
            {
                "event_id": "E686",
                "event_name": "Annual Alumni Meet",
                "date": "2024-02-27",
                "location": "Tokyo, USA"
            },
            {
                "event_id": "E261",
                "event_name": "Annual Alumni Meet",
                "date": "2023-11-22",
                "location": "Boston, Canada"
            },
            {
                "event_id": "E592",
                "event_name": "Annual Alumni Meet",
                "date": "2018-09-17",
                "location": "London, USA"
            }
        ],
        "tags": [
            "Entrepreneur",
            "AI Enthusiast"
        ]
    },
    {
        "id": "97",
        "name": "Alice Johnson",
        "email": "user97@example.com",
        "phone": "+93-600-3078",
        "location": {
            "city": "Boston",
            "state": "",
            "country": "Germany"
        },
        "graduation_year": 2022,
        "degree": "B.Tech",
        "stream": "Electronics",
        "skills": [
            "Finance Analysis",
            "Python"
        ],
        "current_employment": {
            "designation": "Financial Analyst",
            "company": "Global Analytics",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E575",
                "event_name": "Annual Alumni Meet",
                "date": "2020-05-21",
                "location": "Tokyo, Germany"
            },
            {
                "event_id": "E288",
                "event_name": "Research Conference",
                "date": "2018-01-05",
                "location": "Sydney, Germany"
            },
            {
                "event_id": "E669",
                "event_name": "Research Conference",
                "date": "2015-08-18",
                "location": "London, Australia"
            }
        ],
        "tags": [
            "Entrepreneur"
        ]
    },
    {
        "id": "98",
        "name": "Emily Davis",
        "email": "user98@example.com",
        "phone": "+90-567-2817",
        "location": {
            "city": "Toronto",
            "state": "CA",
            "country": "UK"
        },
        "graduation_year": 2022,
        "degree": "MBA",
        "stream": "Electronics",
        "skills": [
            "Cloud Computing",
            "Data Analysis"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "Global Analytics",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E607",
                "event_name": "Finance Workshop",
                "date": "2019-07-07",
                "location": "London, UK"
            },
            {
                "event_id": "E978",
                "event_name": "Research Conference",
                "date": "2020-02-20",
                "location": "San Francisco, Canada"
            },
            {
                "event_id": "E983",
                "event_name": "Finance Workshop",
                "date": "2021-05-06",
                "location": "Tokyo, Canada"
            }
        ],
        "tags": [
            "Mentor",
            "Developer"
        ]
    },
    {
        "id": "99",
        "name": "Sophia Martinez",
        "email": "user99@example.com",
        "phone": "+32-343-9065",
        "location": {
            "city": "Tokyo",
            "state": "",
            "country": "USA"
        },
        "graduation_year": 2009,
        "degree": "PhD",
        "stream": "Computer Science",
        "skills": [
            "Risk Management",
            "Python",
            "Finance Analysis",
            "Machine Learning",
            "AI"
        ],
        "current_employment": {
            "designation": "Data Scientist",
            "company": "Innovatech",
            "location": "London"
        },
        "events_attended": [
            {
                "event_id": "E392",
                "event_name": "Tech Symposium",
                "date": "2019-09-26",
                "location": "Toronto, UK"
            }
        ],
        "tags": [
            "Developer",
            "Researcher"
        ]
    },
    {
        "id": "100",
        "name": "Michael Wilson",
        "email": "user100@example.com",
        "phone": "+69-209-6233",
        "location": {
            "city": "London",
            "state": "MA",
            "country": "Japan"
        },
        "graduation_year": 2001,
        "degree": "B.Tech",
        "stream": "Electronics",
        "skills": [
            "Machine Learning",
            "Data Analysis",
            "Cloud Computing"
        ],
        "current_employment": {
            "designation": "Software Engineer",
            "company": "TechCorp",
            "location": "Sydney"
        },
        "events_attended": [
            {
                "event_id": "E545",
                "event_name": "Tech Symposium",
                "date": "2022-06-28",
                "location": "Sydney, Canada"
            },
            {
                "event_id": "E727",
                "event_name": "Tech Symposium",
                "date": "2023-09-08",
                "location": "San Francisco, USA"
            },
            {
                "event_id": "E238",
                "event_name": "Annual Alumni Meet",
                "date": "2024-01-07",
                "location": "Sydney, Canada"
            }
        ],
        "tags": [
            "Finance Specialist"
        ]
    }
]

# Prepare data for upsertion
batch_size = 10  # Adjust batch size as needed
formatted_batches = []

for item in tqdm(dataset, desc="Processing data"):
    # Create a single string for embedding
    text_to_embed = f"{item['name']} {item['email']} {item['phone']} {item['location']['city']} {item['location']['country']} {item['degree']} {item['stream']} {' '.join(item['skills'])} {' '.join([event['event_name'] for event in item['events_attended']])} {' '.join(item['tags'])}"

    # Encode the text to generate a vector
    vector = model.encode(text_to_embed).tolist()

    # Prepare the data for upsert
    vector_data = {
        "id": item["id"],
        "values": vector
    }
    formatted_batches.append(vector_data)

    # Upsert in batches
    if len(formatted_batches) >= batch_size:
        index.upsert(formatted_batches)
        formatted_batches = []

# Upsert any remaining data
if formatted_batches:
    index.upsert(formatted_batches)


Processing data:   0%|          | 0/100 [00:00<?, ?it/s]

## Making Queries

Now that our index is populated we can begin making queries. We are performing a semantic search for *similar questions*, so we should embed and search with another question. Let's begin.

In [27]:
import torch
from sentence_transformers import SentenceTransformer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

Now let's query.

In [28]:
query = "any mentor available?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(vector=xq, top_k=10, include_metadata=True)
xc

{'matches': [{'id': '69', 'score': 0.424207687, 'values': []},
             {'id': '73', 'score': 0.397780299, 'values': []},
             {'id': '34', 'score': 0.388758063, 'values': []},
             {'id': '32', 'score': 0.386975408, 'values': []},
             {'id': '94', 'score': 0.366522908, 'values': []},
             {'id': '70', 'score': 0.361086667, 'values': []},
             {'id': '23', 'score': 0.359037042, 'values': []},
             {'id': '45', 'score': 0.35007295, 'values': []},
             {'id': '59', 'score': 0.340013146, 'values': []},
             {'id': '42', 'score': 0.338206351, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In the returned response `xc` we can see the most relevant questions to our particular query — we don't have any exact matches but we can see that the returned questions are similar in the topics they are asking about. We can reformat this response to be a little easier to read:

In [24]:
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['values']}")

0.4: []
0.38: []
0.37: []
0.36: []
0.35: []
0.35: []
0.33: []
0.32: []
0.32: []
0.32: []


These are good results, let's try and modify the words being used to see if we still surface similar results.

In [20]:
query = "which are the best places one should visit in kolkata?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(vector=xq, top_k=5, include_metadata=True)
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

1.0:  Which are the best places one should visit in Kolkata?
0.82:  Is Kolkata a good place to live?
0.79:  Which is the best food street in Kolkata?
0.75:  I am a TCS employee and I am moving to Kolkata around mid September. Where should I live?
0.75:  Do you really think Kolkata is good place to live?


Here we used different terms in our query than that of the returned documents. We substituted **"city"** for **"metropolis"** and **"populated"** for **"number of people"**.

Despite these very different terms and *lack* of term overlap between query and returned documents — we get highly relevant results — this is the power of *semantic search*.

You can go ahead and ask more questions above. When you're done, delete the index to save resources:

In [ ]:
pc.delete_index(index_name)

---